Purpose: Demonstrate how leakage can be detected using a data randomization process.  
Process: Create fake data using bernoulli GLM data-generating process (DGP). Intentionally leak information and then show that this code helps detect the leakage.

In [1]:
import pandas as pd
import sys
import numpy as np
import os
import math
from sklearn import linear_model
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn import metrics 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')

In [2]:
# Set seed
np.random.seed(0)

In [35]:
def randomize(filepath, index_col = None):
    '''
    Randomize column values of a file. Each column is randomized independently.
    
    Inputs:
        filepath (str): path to file to randomize; may be of type csv or txt
        index_col (str): optional name of column to use as index; will not be randomized
    
    Outputs:
        df (dataframe): dataframe representation of randomized data
        Output file will be generated, named as original file name + "_randomized"
    '''
    
    # Treat csv and txt differently
    filename, file_extension = os.path.splitext(filepath)
    if file_extension == '.csv':
        sep = ","
    elif file_extension == '.txt':
        sep = "\t"
        
    # Randomize each column
    df = pd.read_csv(filepath, sep = sep, index_col = index_col) 
    cols = df.columns
    for col in cols:
        print('... Randomizing column ' + col)
        df[col] = np.random.permutation(df[col])
        
    # Print to new csv or txt
    new_file = filename + '_randomized' + file_extension
    df.to_csv(new_file)   
    
    return df

In [4]:
def generate_data(n_rows, m_columns, threshold, betas, intercept):
    '''
    Generate fake data for a classification task, based on data generating process where:
    - Features are drawn iid from standard normal distribution
    - Regression equation: y = intercept + beta_1 * x1 + .. + beta_m * xm + error
    - Error is drawn from standard normal distribution
    - Outcome is 1 if p value from logistic function exceeds threshold, 0 otherwise
    
    Inputs:
        n_rows (int): number of observations to generate
        m_columns (int): number of features to generate
        betas (list): list of numerical values to use as coefficients; length must equal m_columns, 
            i.e. don't include intercept
        intercept (int): constant number to use as intercept of regression equation
        
    Outputs:
        X (dataframe): matrix of feature variables
        y (series): outcome variable
    '''
    print("Generating {} observations...".format(n_rows))
    
    # generate column names
    colnames = []
    for i in range(m_columns):
        colnames.append('x' + str(i + 1))
    
    # generate fake data
    df = pd.DataFrame(np.random.normal(size=(n_rows, m_columns)), columns = colnames)
    df.insert(loc = 0, column = 'intercept', value = intercept)
    betas = [1] + betas
    df['z'] = np.multiply(np.array(betas), df).sum(axis=1) + np.random.normal()
    df['pr'] = df['z'].apply(lambda x: 1/(1 + math.exp(x)))
    df['outcome'] = np.where(df['pr'] > threshold, 1, 0)
    df.index.name = 'idx'
    df.to_csv('fake_data.csv')
    
    return df

In [39]:
def run_logreg(X_train, y_train, X_test, y_test):
    
    print("Baseline accuracy: " + str(round(1 - np.mean(y_train), 2)))
    
    # Create a logistic regression object
    logreg = linear_model.LogisticRegression()

    # Train model with training set
    logreg.fit(X_train, y_train)

    # Make predictions with test set
    y_pred = logreg.predict(X_test)

    # Save probabilities
    probs = logreg.predict_proba(X_test)

    # Print accuracy and AUC
    print("Accuracy: " + str(metrics.accuracy_score(y_test, y_pred)))
    print("AUC Score: " + str(metrics.roc_auc_score(y_test, probs[:, 1])))

    # Print confusion matrix
    print(metrics.confusion_matrix(y_test, y_pred))
    
    return logreg

In [28]:
def logreg_pipeline(df):
    '''
    Put raw inputs through basic machine learning pipeline.
    
    Inputs:
        dataframe: Pandas dataframe representing raw input file
    
    OUtputs:
        logistic regression model object
    '''
    # Extract features and outcome from dataframe
    X = df.drop(['z', 'pr', 'outcome'], axis = 1)
    y = df['outcome']
    
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
   
    return run_logreg(X_train, y_train, X_test, y_test)

### Case 1: Stable DGP (same parameters and threshold over time)  
Non-temporal example with stable DGP

Run original input file through pipeline.

In [40]:
df = generate_data(1000, 5, .8, [1,2,3,4,5], 1) # csv named fake_data.csv is generated from this
logreg_pipeline(df)

Generating 1000 observations...
Baseline accuracy: 0.65
Accuracy: 0.984848484848
AUC Score: 0.999765853659
[[205   0]
 [  5 120]]


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Put the input file through randomization process, then run it through the pipeline. Model performs no better than random, as expected.

In [41]:
df_random = randomize('fake_data.csv', 'idx')
logreg_pipeline(df_random)

... Randomizing column intercept
... Randomizing column x1
... Randomizing column x2
... Randomizing column x3
... Randomizing column x4
... Randomizing column x5
... Randomizing column z
... Randomizing column pr
... Randomizing column outcome
Baseline accuracy: 0.64
Accuracy: 0.642424242424
AUC Score: 0.451630956188
[[212   0]
 [118   0]]


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Try introducing an obvious case of data leakage by using the probability column as a feature. Run this through the pipeline. Model performs well, as expected.

In [56]:
df['leaky_col'] = df['pr']
logreg_pipeline(df)

Baseline accuracy: 0.62
Accuracy: 0.981818181818
AUC Score: 0.999486762756
[[222   5]
 [  1 102]]


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Put the input file through randomization process. Again, introduce data leakage into the pipeline. Model performs better than random, thus suggesting the existence of data leakage.

In [58]:
df_random = randomize('fake_data.csv', 'idx')
df['leaky_col'] = df['pr']
logreg_pipeline(df)

... Randomizing column intercept
... Randomizing column x1
... Randomizing column x2
... Randomizing column x3
... Randomizing column x4
... Randomizing column x5
... Randomizing column z
... Randomizing column pr
... Randomizing column outcome
Baseline accuracy: 0.64
Accuracy: 0.990909090909
AUC Score: 0.999676440849
[[213   2]
 [  1 114]]


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)